## 1. ИНН и названия ВУЗов 
Реестр организаций, осуществляющих образовательную деятельность по имеющим государственную аккредитацию образовательным программам http://obrnadzor.gov.ru/ru/opendata/7701537808-RAOO/

In [1]:
import xml.etree.ElementTree as et 
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
xml_file_name = 'obrnadzorgovru-data-20201119-structure-20160713.xml'
xml_file_path = '..\\data\\external\\' + xml_file_name

In [ ]:
tree = et.parse(xml_file_path)
root = tree.getroot()

In [ ]:
print('Всего организаций', len(root[0]))

## 1.1. Общие данные
- StatusName - Текущий статус свидетельства
- TypeName - Вид свидетельства
- RegionName - Субъект РФ
- RegionCode - Код Субъекта РФ

In [ ]:
rows = []
for i in root.findall("./Certificates/Certificate"):
    status_name = getattr(i.find('StatusName'), 'text', None)
    region_name = getattr(i.find('RegionName'), 'text', None)
    region_code = getattr(i.find('RegionCode'), 'text', None)
    rows.append({'status_name': status_name,
                 'region_name': region_name,
                 'region_code': region_code})

In [ ]:
df_cols = ['status_name', 'region_name', 'region_code']
out_df_certificate = pd.DataFrame(rows, columns=df_cols)

In [ ]:
out_df_certificate.head()

## 1.2. Данные из раздела ActualEducationOrganization

- FullName - Полное наименование
- PostAddress - Юридический адрес организации
- INN - ИНН
- FormName - Организационно правовая форма

In [ ]:
rows = []
actual_path = "./Certificates/Certificate/ActualEducationOrganization"
for i in root.findall(actual_path):
    full_name = getattr(i.find('FullName'), 'text', None)
    post_address = getattr(i.find('PostAddress'), 'text', None)
    ogrn = getattr(i.find('OGRN'), 'text', None)
    inn = getattr(i.find('INN'), 'text', None)
    form_name = getattr(i.find('FormName'), 'text', None)
    rows.append({'full_name': full_name,
                 'is_branch': is_branch,
                 'post_address': post_address,
                 'inn': inn,
                 'form_name': form_name})

In [ ]:
df_cols = ['full_name',
           'post_address',
           'inn',
           'form_name']
out_df_actual_education_organization = pd.DataFrame(rows, columns=df_cols)

In [ ]:
out_df_actual_education_organization.head()

## 1.3. Объединим

In [ ]:
df = pd.concat((out_df_certificate,
                out_df_actual_education_organization),
               axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

## 1.4. Выберем только "status_name" == "Действующее" и "full_name" содержит "высшего"

In [ ]:
df['status_name'].unique()

In [ ]:
acting = df["status_name"] == "Действующее"
higher_education = df["full_name"].str.contains('высшего')

In [ ]:
df_vuz = df[acting & higher_education].drop(['status_name'], axis=1)

In [ ]:
df_vuz.info()

In [ ]:
df_vuz.head()

In [ ]:
print('Всего ВУЗов', len(df_vuz))

## 1.5. Посмотрим на организационно правовую форму (столбец form_name)

In [ ]:
df_vuz['form_name'].unique()

Видно, что столбец form_name заполнен небрежно и не полностью. В то же время наименование образовательной организации должно содержать указание на ее организационно-правовую форму и тип образовательной организации (согласно http://273-фз.рф/zakonodatelstvo/federalnyy-zakon-ot-29-dekabrya-2012-g-no-273-fz-ob-obrazovanii-v-rf). Поэтому удалим form_name. И извлечем нужную информацию из full_name.

In [ ]:
df_vuz.drop(['form_name'], axis=1, inplace=True)

## 1.6. Извлечем информацию о категориях ВУЗов из full_name

In [ ]:
vuz_type_set = set()
for name in df_vuz['full_name']:
    name = name.lower()
    name = name.replace('"', '«')
    vuz_type = name.split('«')
    vuz_type = vuz_type[0].split('высшего образования')
    # print(vuz_type[0])
    vuz_type = vuz_type[0].split(' ')
    for word in vuz_type:
        vuz_type_set.add(word)

bug_word_list = ['-', '(славянский)',
                 '', 'астраханской',
                 'воронежский', 'высшего',
                 'кыргызско-российский', 'красного',
                 'области', 'областное',
                 'образования', 'образовательная',
                 'образовательное', 'организация',
                 'ордена', 'профессионального',
                 'профсоюзов', 'российско-армянский',
                 'санкт-петербургское', 'славянский',
                 'среднего', 'тамбовское',
                 'трудового', '–',
                 'знамени', 'высоких',
                 'университет', 'институт',
                 'учреждение']

for word in bug_word_list:
    vuz_type_set.remove(word)

In [ ]:
vuz_type_set

In [ ]:
avtonomnaya = ['автономная',  'автономное']
akkreditovannoye = ['аккредитованное']
byudzhetnoye = ['бюджетное']
voyennoye = ['военное']
gosudarstvennoye = ['государственное']
religioznaya = ['духовная', 'религиозная']
inclusive = ['инклюзивного']
kazennoye = ['казенное']
kultury = ['культуры']
mezhgosudarstvennoye = ['межгосударственное']
munitsipalnoye = ['муниципальное']
negosudarstvennaya = ['негосударственная', 'негосударственное']
nekommercheskaya = ['некоммерческая', 'некоммерческое']
technologies = ['технологий']
federalnoye = ['федеральное']
chastnoye = ['частное']

In [ ]:
categories = [avtonomnaya, akkreditovannoye,
              byudzhetnoye, voyennoye,
              gosudarstvennoye, religioznaya,
              inclusive, kazennoye, kultury,
              mezhgosudarstvennoye, munitsipalnoye,
              negosudarstvennaya, nekommercheskaya,
              technologies, federalnoye,
              chastnoye]

In [ ]:
categories_names = ['is_avtonomnaya', 'is_akkreditovannoye',
                    'is_byudzhetnoye', 'is_voyennoye',
                    'is_gosudarstvennoye', 'is_religioznaya',
                    'is_inclusive', 'is_kazennoye', 'is_kultury',
                    'is_mezhgosudarstvennoye', 'is_munitsipalnoye',
                    'is_negosudarstvennaya', 'is_nekommercheskaya',
                    'is_technologies', 'is_federalnoye',
                    'is_chastnoye']

In [ ]:
for cats, cats_name in zip(categories, categories_names):
    mask = df_vuz["full_name"].str.contains(cats[0])
    if len(cats) > 1:
        mask = mask + df_vuz["full_name"].str.contains(cats[1])
    df_vuz[cats_name] = mask

In [ ]:
df_vuz[categories_names].head()

## 1.7. Визуализируем связь между выделенными категориями

In [ ]:
X = df_vuz[categories_names]

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(abs(X.corr().round(2)),
            xticklabels=X.corr().columns,
            yticklabels=X.corr().columns,
            cmap='RdYlGn',
            center=0,
            annot=True)
plt.title('Коррелограмма типов ВУЗов', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('..\\reports\\figures\\vuz-types-correlogram.png', dpi=100)
plt.show()

## 1.8. Посчитаем, сколько вузов каждой категории

In [ ]:
X.sum()

In [ ]:
df_vuz.head()